# Проект по SQL

Цели проекта: 
- анализ купленной базы данных с книгами,
- получение представления о количестве и качестве имеющихся в ней книг, отзывов и обзоров.


## Исследуем таблицы — выведем первые строки

In [ ]:
query = '''
           SELECT *
           FROM books
           LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Содержит данные о книгах:

    book_id — идентификатор книги;
    author_id — идентификатор автора;
    title — название книги;
    num_pages — количество страниц;
    publication_date — дата публикации книги;
    publisher_id — идентификатор издателя.


In [ ]:
query = '''
           SELECT *
           FROM authors
           LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Содержит данные об авторах:
    
    author_id — идентификатор автора;
    author — имя автора.


In [ ]:
query = '''
           SELECT *
           FROM publishers
           LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержит данные об издательствах:
    
    publisher_id — идентификатор издательства;    
    publisher — название издательства;


In [ ]:
query = '''
           SELECT *
           FROM ratings
           LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Содержит данные о пользовательских оценках книг:

    rating_id — идентификатор оценки;
    book_id — идентификатор книги;
    username — имя пользователя, оставившего оценку;
    rating — оценка книги.


In [ ]:
query = '''
           SELECT *
           FROM reviews
           LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержит данные о пользовательских обзорах:

    review_id — идентификатор обзора;
    book_id — идентификатор книги;
    username — имя автора обзора;
    text — текст обзора.

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [ ]:
query = '''
           SELECT COUNT(*)
           FROM books
           WHERE publication_date > '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


In [ ]:
query = '''
           SELECT COUNT(*)
           FROM books
        '''

pd.io.sql.read_sql(query, con = engine) 

,count
0,1000


После 1 января 2000 года вышло 819 книгю Всего в базе книг - 1000 книг

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

Объединим таблицы books, reviews, ratings.

In [ ]:
query = '''
            SELECT books.book_id,
            books.title,
            COUNT(DISTINCT review_id) AS count_reviews,
            AVG(rating) AS avg_rating
            FROM books
            LEFT JOIN  reviews ON books.book_id = reviews.book_id
            LEFT JOIN ratings ON books.book_id = ratings.book_id
            GROUP BY books.book_id
            ORDER BY count_reviews DESC
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Мы получили список книг с количествами обзоров и средней оценкой. Больше всего обзоров на книгу Сумерки

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Объединим таблицы books и publishers по столбцу publisher_id, выберем книги больше 50 страниц, сгруппируем по publisher_id, отсортируем по количеству книг.

In [ ]:
query = '''
            SELECT publishers.publisher_id,
            publishers.publisher,
            COUNT(DISTINCT book_id) AS count_book
            FROM books
            LEFT JOIN  publishers ON books.publisher_id = publishers.publisher_id
            WHERE books.num_pages >50
            GROUP BY publishers.publisher_id
            ORDER BY count_book DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_book
0,212,Penguin Books,42


Издательство Penguin Books выпустило больше всех книг с количеством страниц больше 50	

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [ ]:
query = '''
            WITH book_50 AS
                (SELECT books.book_id,
                books.author_id,
                COUNT(ratings.rating),
                AVG(ratings.rating)
                FROM books
                LEFT JOIN  ratings ON books.book_id = ratings.book_id
                GROUP BY books.book_id
                HAVING COUNT(ratings.rating) > 50)
                
            SELECT authors.author_id,
            authors.author, 
            AVG(book_50.avg) AS avg_rating_autor    
            FROM book_50
            LEFT JOIN  authors ON book_50.author_id = authors.author_id
            GROUP BY authors.author_id
            ORDER BY avg_rating_autor DESC
            LIMIT 1    
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating_autor
0,236,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Сформируем временную таблицу, в которую отберем пользователей, которые поставили больше 50 оценок. Посчитаем для этих пользователей количество обзоров, для этого объединим временную таблицу и таблицу reviews, чтобы в ней были только пользователи из временной таблицы. И посчитаем среднее по получившейся выборке

In [ ]:
query = '''  
            SELECT AVG(count)
            FROM (WITH us_50 AS (SELECT username, 
                  COUNT(rating)
                  FROM ratings
                  GROUP BY username
                  HAVING COUNT(rating) > 50)
                
                  SELECT COUNT(review_id)
                  FROM us_50
                  LEFT JOIN reviews ON us_50.username = reviews.username
                  GROUP BY reviews.username) as avg
            

       '''

pd.io.sql.read_sql(query, con = engine)


,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33